## Homework 4: StyleGAN 💇🏻‍♂️

С момента своего появления в 2014 году генератор в GAN-ах работал как черный ящик. В традиционных архитектурах скрытый вектор $\mathbf{z}$ подавался в первый слой сети, и, проходя через цепочку сверток, превращался в изображение. Хотя качество изображений росло, исследователи столкнулись с проблемой так называемой **запутанности** (**entanglement**) скрытого пространства. То есть попытка изменить одну характеристику лица неизбежно влекла за собой нежелательные изменения других атрибутов.

Прорывом, предшествующим появлению `StyleGAN`, стала идея **прогрессивного роста** (**Progressive Growing**), предложенная командой `NVIDIA` в модели [ProGAN](https://arxiv.org/pdf/1710.10196). Идея заключалась в том, чтобы начинать обучение с изображений низкого разрешения, постепенно добавляя новые слои и повышая детализацию. Это позволило стабилизировать обучение и достичь высокого разрешения $1024\times 1024$. Однако, несмотря на высокое качество, в `ProGAN` оставалась проблема запутанности скрытого пространства.

Авторы [StyleGAN](https://arxiv.org/pdf/1812.04948) оставили проверенную идею `ProGAN`, но радикально изменили устройство генератора. Вдохновившись методами **переноса стиля** (**Style Transfer**), авторы предложили полностью отказаться от подачи скрытого вектора $\mathbf{z}$ в первый слой нейросети. Вместо этого они предложили начинать с обучаемого тензора,а управление процессом генерации осуществлять через отдельную **сеть отображения** (**Mapping Network**), которая преобразует исходный латентный вектор из искривленного пространства $\mathcal{Z}$ в промежуточное, распутанное пространство $\mathcal{W}$.

### Задание

Вам предстоит реализовать модель `StyleGAN` и обучить модель на датасете `CelebA HQ` для решения задачи генерации лиц.

За выполнение домашнего задания можно получить до **11 баллов**. Для части заданий мы написали для вас скелет. Заполните в них пропуски, выделенные с помощью `...`.

In [ ]:
import copy
import os
import random
from collections import OrderedDict
import zipfile

import gdown
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image 
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torchvision.utils import make_grid

from IPython.display import clear_output

### Задание 1: Dataset (0.5 балла)

Для обучения нашей модели мы будем использовать датасет `CelebA HQ`, который содержит $30$ тыс. изображений лиц знаменитостей в разрешении $128\times 128$ пикселей.

**Ваша задача**:

- Скачать и распаковать архив с датасетом

- Создать преобразования для всех изображений: 
    - преобразовать в тензор (`ToTensor`).
    - сделать нормализацию (`Normalize`)

- Реализовать и создать класс `CelebADataset`.

- Создать `train_Loader` для всего датасета.

In [ ]:
def download_and_unzip(file_id, archive_path, target_dir):
    if os.path.isdir(target_dir) and os.listdir(target_dir):
        print(f"Directory '{target_dir}' is not empty. Skipping download.")

    os.makedirs(os.path.dirname(archive_path), exist_ok=True)
    os.makedirs(target_dir, exist_ok=True)

    print(f"Downloading archive ID {file_id} to {archive_path}...")
    gdown.download(id=file_id, output=archive_path, quiet=False, fuzzy=True)
    print("Download complete.")
    
    print(f"Unzipping archive '{archive_path}' to '{target_dir}'...")
    with zipfile.ZipFile(archive_path, 'r') as zip_ref:
        zip_ref.extractall(target_dir) 
    print("Unzipping complete.")
    
    os.remove(archive_path)
    print(f"Archive '{archive_path}' deleted.")
    
DATASET_ID = "1bnYNmmMpb0eXr028qDzB9jHNUwJTCghl" 
TEMP_DIR = "data/temp" 
ARCHIVE_PATH = os.path.join(TEMP_DIR, "celeba_128.zip")
FINAL_IMAGE_DIR = "data/celeba_hq/" 

download_and_unzip(DATASET_ID, ARCHIVE_PATH, FINAL_IMAGE_DIR)

In [ ]:
class CelebADataset(Dataset):
    def __init__(self, root_dir: str, transform=None):
        self.transform = transform
        self.image_paths = ...
        
    def __len__(self):
        ...
    def __getitem__(self, idx):
        ...

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ 
# Your code here

train_dataset = ...
train_loader = ...

Взглянем на наши данные.

In [ ]:
indices = random.sample(range(len(train_dataset)), 20)
fig, axes = plt.subplots(4, 5, figsize=(15, 12))
axes = axes.flatten()

for i, idx in enumerate(indices):
    ax = axes[i]
    img_tensor = train_dataset[idx]
    
    img_display = img_tensor.permute(1, 2, 0) * 0.5 + 0.5
    ax.imshow(img_display)
    ax.axis('off')

plt.tight_layout()
plt.show()

### Задание 2: Equalized Learning Rate (0.5 балла)

В стандартных нейросетях мы обычно используем инициализацию весов один раз перед началом обучения, чтобы привести дисперсию активаций к разумным пределам. Однако в `GAN`-ах, веса могут со временем сильно изменяться, что приводит к нестабильности градиентов.

Авторы ProGAN предложили использовать **Equalized Learning Rate** — механизм, который динамически контролирует масштаб весов в процессе работы сети.

Основным фактором нестабильности является $\text{fan\_in}$, или количество входных связей для каждого нейрона. Чем больше $\text{fan\_in}$, тем сильнее «взрывается» сигнал. Для компенсации этого эффекта было предложено использовать коэффициент масштаба $c$:

$$c = \sqrt{2 / \text{fan\_in}}$$

На практике механизм работает так:

1. Инициализируем веса из нормального распределения $\mathcal{N}(0, 1)$.

2. Не меняем веса при инициализации, а вычисляем масштабирующий коэффициент $c$ на основе $\text{fan\_in}$.

3. Во время каждого прохода мы умножаем наши веса на этот коэффициент

Реализуйте механизм `Equalized Learning Rate` в линейном и свёрточном слоях

In [ ]:
class EqualizedLinear(nn.Module):
    """
    Linear layer with Equalized Learning Rate
    Used in the Mapping Network and for Style Modulation
    Args:
        input_size: Size of input sample
        output_size: Size of output sample
        gain: Activation scaling factor, usually sqrt(2)
        lrmul: Learning rate multiplier
    """
    def __init__(self, input_size, output_size, gain=2**0.5, lrmul=1):
        super().__init__()
        
        # Calculate He initialization constant: gain * sqrt(1 / fan_in)
        # For Linear, fan_in = input_size
        he_std = ... 
        
        self.w_mul = he_std * lrmul
        self.b_mul = lrmul
        init_std = 1.0 / lrmul
            
        # Initialize weights with Standard Normal Distribution N(0, 1) scaled by init_std
        self.weight = nn.Parameter(...) 
        self.bias = nn.Parameter(torch.zeros(output_size))

    def forward(self, x):
        # Scale the weights and bias using self.w_mul and self.b_mul before passing to F.linear 
        scaled_weight = ...
        scaled_bias = ...
        return F.linear(x, scaled_weight, scaled_bias)
    

class EqualizedConv2d(nn.Module):
    """
    Convolutional layer with Equalized Learning Rate (EqLR)
    Args:
        input_channels: Number of input channels
        output_channels: Number of output channels
        kernel_size: Kernel size 
        stride: Stride of the convolution
        gain: Activation scaling factor
        lrmul: Learning rate multiplier
        padding: Zero-padding
    """
    def __init__(self, input_channels, output_channels, kernel_size, stride=1, gain=2**0.5, lrmul=1.0, padding=None):
        super().__init__()
        
        # Calculate He initialization constant
        # For Conv2d, fan_in = input_channels * kernel_width * kernel_height
        fan_in = ...
        he_std = ...
        
        self.kernel_size = kernel_size
        self.w_mul = he_std * lrmul
        self.b_mul = lrmul
        init_std = 1.0 / lrmul

        # Initialize self.weight with N(0, 1) scaled by init_std
        self.weight = nn.Parameter(...)
        self.bias = nn.Parameter(torch.zeros(output_channels))
            
        self.stride = stride
        self.padding = padding if padding is not None else kernel_size // 2

    def forward(self, x):
        # Scale weights by self.w_mul and bias by self.b_mul
        scaled_weight = ...
        scaled_bias = ...
        return F.conv2d(
            x,
            scaled_weight, 
            scaled_bias,
            stride=self.stride,
            padding=self.padding
        )

### Задание 3: Pixel Normalization Layer, NoiseLayer (1 балл)

#### Pixel Normalization Layer

Изначально архитектуры `GAN` столкнулись с тем, что `Batch Normalization` не работала должным образом из-за того, что размеры батчей часто были слишком малы, а статистика по ним — слишком шумной.

Авторы `ProGAN` предложили использовать **нормализацию по пикселям** (**Pixel Normalization**).

Вместо того чтобы нормализовать данные по всему батчу, мы нормализуем $L_2$-норму вектора признаков вдоль канала для каждого отдельного пикселя. То есть, для каждого $(x, y)$ пикселя мы берем его вектор признаков по $C$ каналам и приводим его длину к единице:

$$b_{x,y} = \frac{a_{x,y}}{\sqrt{\frac{1}{C} \sum_{j=0}^{C-1} (a_{x,y}^j)^2 + \epsilon}}$$

Этот метод стабилизирует модель и особенно важен в самом начале генератора, в `Mapping Network`, где он применяется к входному вектору $\mathbf{z}$.

Реализуйте метод `forward` для слоя `PixelNormLayer`. 

In [ ]:
class PixelNormLayer(nn.Module):
    """
    Pixel Normalization
    Used in the Mapping Network to normalize the magnitude of feature vectors
    """
    def __init__(self, epsilon=1e-8):
        super().__init__()
        self.epsilon = epsilon

    def forward(self, x):
        
        # Calculate Mean Square across the channel dimension (dim=1)
        ms = ...

        # Calculate the inverse Root Mean Square (rRMS), use torch.rsqrt() for computational efficiency

        rrms = ...
        
        # Apply rRMS to the input tensor x
        x_norm = ...
        
        return x_norm

#### NoiseLayer

В предыдущих архитектурах `GAN` вектор $\mathbf{z}$ отвечал сразу и за глобальные характеристики, и за мелкие детали. Поэтому, когда нужно было добавить локальные изменения, сеть пыталась получить их из глобальных характеристик. Это приводило к тому, что мелкие детали выглядели искусственными и не подходили к геометрии лица.

В `StyleGAN` решили эту проблему, введя специальный **слой шума** (**Noise Layer**) после каждого сверточного слоя в сети синтеза.

Это позволило генератору разделить обязанности:

- **Вектор стиля** $\mathbf{w}$: контролирует высокоуровневые атрибуты (поза, возраст)

- **Слой шума**: контролирует мелкие детали, которые не влияют на общую картину

Шум масштабируется с помощью обучаемых весов `self.weight`. Это позволяет сети решать, сколько случайности нужно добавить для каждого конкретного признака.

Ваша задача — реализовать метод `forward` в слое `NoiseLayer`.

In [ ]:
class NoiseLayer(nn.Module):
    """
    Injects per-pixel Gaussian noise, scaled by a learned per-channel weight.
    """
    def __init__(self, channels):
        super().__init__()
        self.weight = nn.Parameter(torch.zeros(channels))
        self.noise = None 

    def forward(self, x, noise=None):
        # x shape: [B, C, H, W]
        
        if noise is None and self.noise is None:
            # Create the noise tensor using torch.randn (noise must be 1-channel for broadcasting)
            noise = ...

        elif noise is None:
            noise = self.noise
        
        # Scale the noise by the learned weight and add to features
       
        return ...

### Задание 4: AdaIN (1 балл)

Вместо того чтобы просто подавать вектор стиля $\mathbf{w}$ в начале, `StyleGAN` использует его для контроля статистики карты признаков на каждом уровне разрешения. Этот процесс регулировки распределения признаков достигается за счет **адаптивной нормализации** (**Adaptive Instance Normalization**, **AdaIN**).

AdaIN состоит из двух этапов, которые мы реализуем с помощью двух классов:

- **Нормализация** $\mathbf{x}_i \to \mathbf{x}'$: Сначала стирается старый стиль карты признаков, причем каждая карта признаков $\mathbf{x}_i$ нормализуется отдельно. Этот этап реализуется с помощью `Instance Normalization` внутри `StyleBlock`.

- **Трансформация** ($\mathbf{w} \to \mathbf{y} \to \mathbf{b}$): Затем мы внедряем новый стиль. Вектор $\mathbf{w}$ проходит через обучаемый линейный слой и превращается в вектор стиля $\mathbf{y} = (y_s, y_b)$. Эти компоненты используются для масштабирования ($y_{s,i}$) и сдвига ($y_{b,i}$) нормализованного вектора $\mathbf{x}'$. Этот этап реализуется в классе `StyleModulation`.

Формула, определяющая эту операцию:

$$\text{AdaIN}(\mathbf{x}_i, \mathbf{y}) = y_{s,i} \frac{\mathbf{x}_i - \mu(\mathbf{x}_i)}{\sigma(\mathbf{x}_i)} + y_{b,i}$$

#### StyleModulation

Этот класс отвечает за финальную трансформацию. Он берет вектор $\mathbf{w}$, преобразует его в параметры стиля $y_{s,i}$ и $y_{b,i}$ и применяет их к нормализованной карте признаков.

Вам необходимо заполнить пропуски в классе `StyleModulation`.

In [ ]:
class StyleModulation(nn.Module):
    """
    Applies the scale (ys) and bias (yb) parameters derived from w.
    This performs the Adaptive part of the AdaIN operation.
    """
    def __init__(self, latent_size, channels): 
        super().__init__()
        self.lin = EqualizedLinear(latent_size, channels * 2, gain=1.0, lrmul=1.0)

    def forward(self, x, latent):
        style = self.lin(latent)
        
        # Reshape style into [B, 2 (scale/bias), C, 1, 1] for broadcasting
        shape = [-1, 2, x.size(1)] + (x.dim() - 2) * [1]
        style = style.view(shape)
        
        # get ys and yb (ys corresponds to style[:, 0], yb to style[:, 1])
        ys = ...
        yb = ...
        
        #Apply the final AdaIN formula
        return ...

#### StyleBlock

Этот класс управляет последовательностью операций в генераторе. Его основная функция — обеспечить правильную последовательность: 

`Noise` $\to$ `Activation` $\to$ `InstanceNorm` $\to$ `StyleAdjustment`. 

`Instance Norm` стирает старый стиль, подготавливая карту признаков к новому стилю.

In [ ]:
class StyleBlock(nn.Module):
    """
    The full synthesis pipeline: Noise -> Activation -> Norm -> StyleAdjustment.
    Args:
        channels: Number of feature channels in the input map
        dlatent_size: Dimensionality of the intermediate latent space
        activation_layer: The activation function to use
    """
    def __init__(self, channels, dlatent_size, activation_layer):
        super().__init__()

        layers = []
        
        layers.append(('noise', NoiseLayer(channels)))
        layers.append(('activation', activation_layer))
        layers.append(('instance_norm', nn.InstanceNorm2d(channels)))

        self.main_ops = nn.Sequential(OrderedDict(layers))
        self.style_mod = StyleModulation(latent_size=dlatent_size, channels=channels) 

    def forward(self, x, dlatents_in_slice=None):
        """
        Processes the feature map and applies style adjustment.
        :param x: The input feature map
        :param dlatents_in_slice: A slice of the W vector containing the style parameters for this layer 
        """
        # Get x_norm using main_ops
        x_norm = ...
        
        # Apply Style Modulation
        x = ...
        
        return x

### Задание 5: Minibatch Standard Deviation (0.5 балла)

Основной проблемой в архитектурах `GAN` является **коллапс мод** (**Mode Collapse**) — генератор находит ограниченный набор изображений, которые успешно обманывают дискриминатор, а затем создаёт новые объекты только из этой моды.

Обычный дискриминатор обрабатывает каждое изображение независимо от других в батче, поэтому он не может оценить, насколько разнообразен весь батч фейковых изображений. 

Идея **Minibatch Standard Deviation** (**MBSD**) была впервые представлена в 2016 году [Salimans et al.](https://arxiv.org/pdf/1606.03498) и была адаптирована для модели `ProGAN`. **MBSD** решает проблему коллапса мод, предоставляя дискриминатору возможность оценивать разнообразие текущего мини-батча.

**MBSD** работает как детектор однообразия, который говорит дискриминатору, насколько сильно отличаются изображения в батче:

- Слой делит мини-батч на небольшие группы изображений

- Для каждого признака $\mathbf{x}$ внутри группы рассчитывается стандартное отклонение $\boldsymbol{\sigma}$ по оси батча. Это показывает, насколько сильно отличается один и тот же признак на разных изображениях

- Все полученные значения $\boldsymbol{\sigma}$ усредняются по пространственным координатам и каналам

- Этот вектор добавляется как новый канал к выходной карте признаков дискриминатора

Вам нужно реализовать шаги для расчета стандартного отклонения $\boldsymbol{\sigma}$, описанные выше.

In [ ]:
class MinibatchStddev(nn.Module):
    """
    Calculates the standard deviation of features across the minibatch group 
    and appends it as a new feature map to discourage mode collapse.
    Args:
        group_size: The number of samples to group together for statistics, must be a divisor of the batch size 
        num_new_features: Number of new feature maps to append
    """
    def __init__(self, group_size=4, num_new_features=1):
        super().__init__()
        self.group_size = group_size
        self.num_new_features = num_new_features

    def forward(self, x):
        b, c, h, w = x.shape
        group_size = min(self.group_size, b)
        
        # Reshape to [G, M, F, C', H, W] where G is group_size, M is Minibatches, F is num_new_features, C' is Internal channels
        y = x.view(group_size, -1, self.num_new_features, c // self.num_new_features, h, w)
        
        # Calculate the standard deviation sqrt(Mean((y - mean(y))^2) + epsilon)
        y_std = ...

        # Average Std Dev over all non-group dimensions (C', H, W)
        y_avg = ...
        
        # Replicate y_avg over the group dimension and expand to [B, F, H, W]
        y_final = ...
        
        #  Concatenate x and y along the channel dimension
        z = ...
        return z

### Задание 6: Truncation Trick (0.5 балла)

Обученная модель лучше всего справляется с генерацией изображений, соответствующих высокой плотности в латентном пространстве, то есть, усредненных или "типичных" лиц. Области с низкой плотностью часто содержат нетипичные, сильно искаженные изображения и артефакты.

**Трюк усечения** (**Truncation Trick**) — это метод, который применяется во время генерации. Его цель — принудительно переместить любой случайно сгенерированный вектор стиля $\mathbf{w}$ из областей с низкой плотностью обратно к среднему вектору стиля $\bar{\mathbf{w}}$. 

С одной стороны, этот механизм повышает качество и уменьшает количество артефактов, а с другой — снижает разнообразие объектов.

Трюк работает следующим образом:

- В процессе обучения мы рассчитываем скользящее среднее $\bar{\mathbf{w}}$ всех сгенерированных векторов стиля $\mathbf{w}$.

- При генерации новый вектор $\mathbf{w}$ смешивается с $\bar{\mathbf{w}}$ с использованием коэффициента $\psi$ (`threshold`):

$$\mathbf{w}' = \bar{\mathbf{w}} + \psi \cdot (\mathbf{w} - \bar{\mathbf{w}})$$

В `StyleGAN` этот трюк применяется только к первым `max_layer` векторам, которые контролируют грубые, глобальные атрибуты.

Ваша задача — реализовать логику **Truncation Trick**.

In [ ]:
class Truncation(nn.Module):
    """
    Implements the Truncation Trick in W-space to bias random samples 
    towards the mean, improving fidelity at the cost of diversity.
    """
    def __init__(self, avg_latent, max_layer=8, threshold=0.7, beta=0.995):
        super().__init__()
        self.max_layer = max_layer
        self.threshold = threshold
        self.beta = beta
        self.register_buffer('avg_latent', avg_latent)

    def update(self, last_avg):
        """
        Calculates the new exponential moving average (EMA) of the W vector.
        :param last_avg: The mean W vector of the current training batch.
        """
        # Implement the EMA : new_avg = beta * old_avg + (1 - beta) * current_avg
        self.avg_latent.copy_(...) 

    def forward(self, x):
        # Apply Truncation formula, use torch.lerp
        interp = ...
        
        # Create the masking tensor (only for layers < max_layer)
        do_trunc = ...
        
        # Apply the mask

        return ...

### Задание 7: Constant Input Block (0.5 балла)

В отличие от предшествующих моделей, где скрытый вектор $\mathbf{z}$ напрямую подавался в первый слой генератора, `StyleGAN` начинает синтез изображения не с шума, а с обучаемой константы. За это отвечает класс `ConstantInputBlock`.

Обучаемая константа `self.const` выполняет роль холста — обучаемого тензора размера $4 \times 4 \times 512$, который даёт начальную пространственную структуру изображения. Поскольку этот холст уже содержит оптимальную базовую информацию, генератору не нужно тратить вычислительные ресурсы на её формирование из вектора $\mathbf{z}$.

Задача `ConstantInputBlock` сводится к тому, чтобы взять этот обучаемый тензор и начать его стилизацию:

- Сначала исходному тензору применяется первый вектор стиля $\mathbf{w}_0$, что задает грубые, высокоуровневые атрибуты изображения

- Затем следует свертка, а после нее — второе добавление вектора стиля $\mathbf{w}_1$

Вам нужно реализовать логику `ConstantInputBlock`, заполнив пропуски в коде.

In [ ]:
class ConstantInputBlock(nn.Module):
    """
    The initial 4x4 block of the StyleGAN Generator. 
    It replaces the direct Z input with a trainable constant
    """
    def __init__(self, 
                 num_channels, 
                 dlatent_size, 
                 activation_layer):
        
        super().__init__()
        
        # Initialize self.const with a tensor of ones
        self.const = nn.Parameter(...) 
        
        # We also need a bias parameter for the constant input
        self.bias = nn.Parameter(...)
        
        self.style_block1 = StyleBlock(
            channels=num_channels, 
            dlatent_size=dlatent_size, 
            activation_layer=activation_layer
        )
        
        self.conv = EqualizedConv2d(num_channels, num_channels, kernel_size=3, padding=1)
        
        self.style_block2 = StyleBlock(
            channels=num_channels, 
            dlatent_size=dlatent_size,  
            activation_layer=activation_layer
        )

    def forward(self, w_slice):
        # w_slice contains the first two W vectors: [B, 2, D]
        batch_size = w_slice.size(0)

        # Expand the constant to match the batch size [1, C, 4, 4] -> [B, C, 4, 4]
        x = ...
        
        # Add the learned bias
        x = ...

        # Apply the first style w_0
        x = ...
        
        # Pass through convolution
        x = ...
        
        # Apply the second style w1
        x = ...
        
        return x

### Задание 8: Synthesis Block (0.5 балла)

`SynthesisBlock` — это основной блок, который принимает карту признаков низкого разрешения и выдает карту признаков вдвое большего размера.

В отличие от `ConstantInputBlock`, который только создает заготовку $4 \times 4$, этот блок является повторяющимся циклом синтеза, состоящим из 3 этапов:

1. **Upsampling**

2. **First Convolution and Style Adjustment**

3. **Second Convolution and Final Style Adjustment**

Заметьте, что блок потребляет два вектора стиля на одно разрешение. Это сделано для того, чтобы увеличить емкость и управляемость сети. Первый вектор стиля контролирует признаки сразу после повышения разрешения, а второй — уже обработанные признаки. 

Благодаря такому **Style Mixing** мы можем смешивать атрибуты, взятые с разных разрешений.

#### Upsample, DownSample

Архитектура `StyleGAN` построена на идее **Progressive Growing**, при которой **генератор** и **дискриминатор** работают на разных уровнях разрешения. А для перехода между этими уровнями необходимо измененять размеры карт признаков:

- **Upsampling**: используется генератором для увеличения размера карты признаков

- **Downsampling**: используется дискриминатором для уменьшения размера карты признаков

Реализуйте функционал увеличения и уменьшения разрешения карт признаков, используя функции из `PyTorch`.

In [ ]:
class SynthesisBlock(nn.Module):
    """
    The main repeating block of the Generator pipeline. 
    Doubles the resolution and consumes two W vectors.
    """
    def __init__(self, in_channels, out_channels, dlatent_size, gain, activation_layer):
        super().__init__()

        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')

        self.conv1 = EqualizedConv2d(in_channels, 
                                     out_channels, 
                                     kernel_size=3, 
                                     padding=1, 
                                     gain=gain
                                     )
        
        self.style_block1 = StyleBlock(channels=out_channels, 
                                       dlatent_size=dlatent_size,  
                                       activation_layer=activation_layer
                                       )
        
        self.conv2 = EqualizedConv2d(out_channels, 
                                     out_channels, 
                                     kernel_size=3, 
                                     padding=1, 
                                     gain=gain
                                     )
        self.style_block2 = StyleBlock(channels=out_channels, 
                                       dlatent_size=dlatent_size, 
                                       activation_layer=activation_layer
                                       )

    def forward(self, x, w_slice):
        
        # Apply upsample to the input x
        x = ... 
        # Apply conv1 to the result
        x = ...
        # Apply self.style_block1, using the first w vector slice
        x = ...
        # Apply self.conv2
        x = ...
        # Apply self.style_block2, using the second w vector slice
        x = ...
        
        return x

### Задание 9: Discriminator Output Block (0.5 балла)

`DiscriminatorOutputBlock` является финальным этапом работы дискриминатора,  выполняя три  действия:

1. Обрабатывает карту признаков через слой `MinibatchStddev` и получает информацию о разнообразии всего батча

2. Выполняет свертку $3 \times 3$, которая объединяет все признаки 

3. Результат свертки проходит через два финальных полносвязных слоя, которые и выдают финальный скор для изображения

Реализуйте метод `forward` в классе `DiscriminatorOutputBlock`.

In [ ]:
class DiscriminatorOutputBlock(nn.Module):
    """
    Final block of the Discriminator. It applies MBSD, the last feature 
    extraction, and converts the 4x4 feature map into a single realism score.

    Args:
            mbstd_group_size (int): Number of samples per group for Minibatch Standard Deviation 
            mbstd_num_features (int): Number of new feature maps to append from MBSD
            in_channels (int): Number of input channels to this block
            intermediate_channels (int): Number of channels in the first fully connected layer
            gain (float): He initialization gain for most layers
            activation_layer (nn.Module): The activation function
            resolution (int): The spatial resolution of the input feature map
            in_channels2 (int, optional): Output channels of the last convolution, defaults to in_channels
            output_features (int): Final output size 
            last_gain (float): Gain for the final output linear layer
    """
    def __init__(self, mbstd_group_size, 
                 mbstd_num_features, 
                 in_channels, 
                 intermediate_channels, 
                 gain, 
                 activation_layer, 
                 resolution=4, 
                 in_channels2=None, 
                 output_features=1, 
                 last_gain=1):
        
        super().__init__()
        
        if in_channels2 is None:
            in_channels2 = in_channels
            
        if mbstd_group_size > 1:
            self.mbstd = MinibatchStddev(mbstd_group_size, mbstd_num_features)
            final_in_channels = in_channels + mbstd_num_features
        else:
            self.mbstd = None
            final_in_channels = in_channels

        self.conv = EqualizedConv2d(final_in_channels, in_channels2, kernel_size=3, padding=1, gain=gain)
        self.act1 = activation_layer
        self.fc1 = EqualizedLinear(in_channels2 * resolution * resolution, intermediate_channels, gain=gain)
        self.act2 = activation_layer
        self.fc2 = EqualizedLinear(intermediate_channels, output_features, gain=last_gain)


    def forward(self, x):    

        if self.mbstd is not None:
            # Apply self.mbstd to the input x
            x = ...
            
        # Apply activation to the result of the convolution
        x = self.act1(self.conv(x))
        
        # Flatten the tensor using torch.flatten()
        x_flat = ...
        
        # Apply the first linear layer, then the second activation
        x = ...
        
        # Return the result of the last linear layer
        return ...

### Задание 10: Discriminator Block (0.5 балла)
`DiscriminatorBlock` — это основной модуль критика:

- Принимает карту признаков высокого разрешения

- Обрабатывает её свертками

- Уменьшает разрешение в 2 раза

Блок состоит из двух сверточных слоев и операции **уменьшения разрешения** (**Downsampling**). Для уменьшения разрешения мы будем использовать `Average Pooling`.

Реализуйте `DiscriminatorBlock`, заполнив пропуски в коде.

In [ ]:
class DiscriminatorBlock(nn.Module):
    """
    The main repeating block of the Discriminator.
    It extracts features and then downsamples the resolution.
    """
    def __init__(self, 
                 in_channels, 
                 out_channels, 
                 gain, 
                 activation_layer):
        
        super().__init__()
        
        self.conv1 = EqualizedConv2d(in_channels, in_channels, kernel_size=3, padding=1, gain=gain)
        self.act1 = activation_layer
     
        self.conv2 = EqualizedConv2d(in_channels, out_channels, kernel_size=3, padding=1, gain=gain)
        self.act2 = activation_layer
        
        # Initialize nn.AvgPool2d
        self.downsample = ...

    def forward(self, x):
        # Pass x through conv1 and activation
        x = ...
        
        # Pass x through conv2 and activation
        x = ...
        
        # Apply downsampling
        x = ...

        return x

### Задание 11: Mapping Network (0.5 балла)

В обычных `GAN`-ах входной вектор $\mathbf{z}$ берется из заданного распределения, $\mathbf{z} \sim \mathcal{N}(\mathbf{0},\mathbf{I})$. 

Авторы `StyleGAN` предложили не подавать $\mathbf{z}$ сразу в модель, а сначала пропустить его через полносвязную сеть — `Mapping Network`, которая преобразует искривленное пространство $\mathcal{Z}$ в промежуточное, «распутанное» пространство $\mathcal{W}$.

Ваша задача — реализовать `Mapping Network`.

In [ ]:
class MappingNetwork(nn.Module):
    """
    Maps the entangled input latent Z to the disentangled intermediate latent w

    Args:
        latent_size: Dim of the input latent vector z
        dlatent_size: Dim of the output style vector w
        num_layers_broadcast: If defined, broadcasts the output w for the synthesis network.
        mapping_layers: Number of layers in the MLP
        mapping_fmaps: Number of hidden channels in the MLP layers
        mapping_lrmul: Learning rate multiplier
    """
    def __init__(self, 
                 latent_size=512,          
                 dlatent_size=512,          
                 num_layers_broadcast=None, 
                 mapping_layers=8,         
                 mapping_fmaps=512,        
                 mapping_lrmul=0.01,
                 **kwargs): 
        
        super().__init__()
        
        self.num_layers_broadcast = num_layers_broadcast
        self.pixel_norm = PixelNormLayer()
        self.act = nn.LeakyReLU(negative_slope=0.2)
        gain = np.sqrt(2)
        
        self.layers = nn.ModuleList()
        
        self.layers.append(EqualizedLinear(
            latent_size, mapping_fmaps, 
            gain=gain, lrmul=mapping_lrmul
        ))
        
        for _ in range(1, mapping_layers - 1):
            self.layers.append(EqualizedLinear(
                mapping_fmaps, mapping_fmaps, 
                gain=gain, lrmul=mapping_lrmul
            ))
            
        self.layers.append(EqualizedLinear(
            mapping_fmaps, dlatent_size, 
            gain=gain, lrmul=mapping_lrmul
        ))

    def forward(self, z):
        
        # Apply PixelNorm to z 
        z = ...
            
        # Run through the MLP
        for layer in self.layers:
            # Apply linear layer and activation
            w = ...
            
        if self.num_layers_broadcast is not None:
            # Unsqueeze and expand 'w' to match the number of synthesis layers
            # Target shape: [Batch, Num_Layers_Broadcast, Dlatent_Size]
            w = ...
            
        return w

### Задание 12: Synthesis Network (1 балл)

Сеть синтеза отвечает за преобразование распутанного вектора стиля $\mathbf{w}$ в итоговое изображение.

Для этого `StyleGAN` унаследовал от `ProGAN` стратегию обучения:

- Обучаем сеть генерировать изображения $4 \times 4$

- Когда обучение стабилизируется, мы добавляем новые блоки $8 \times 8$, $16 \times 16$ и так далее

Процесс роста реализован через две ветки, которые смешиваются с коэффициентом $\alpha$ (**Fade-in**):

- **Residual Branch**: берет результат предыдущего, уже обученного разрешения и просто увеличивает его (**Upsample**), минуя новые слои.

- **Straight Branch**: пропускает данные через новые, обучаемые блоки синтеза.

В начале обучения нового разрешения $\alpha \approx 0$ и мы видим только старый слой. По мере обучения $\alpha \to 1$, и мы переключаемся на новые слои.

Вам нужно собрать `Synthesis Network` из уже реализованных блоков.

In [ ]:
class SynthesisNetwork(nn.Module):
    """
    Constructs the image from w vectors using Progressive Growing mechanics.

    Args:
        dlatent_size: Dim of W
        num_channels: Output image channels
        resolution: Target output resolution
        fmap_base: Base number of feature maps
        fmap_decay: Decay rate for feature maps as resolution increases.
        fmap_max: Maximum number of feature maps.
        nonlinearity: Activation function name
    """
    def __init__(self, 
                 dlatent_size=512, 
                 num_channels=3, 
                 resolution=128,
                 fmap_base=8192, 
                 fmap_decay=1.0, 
                 fmap_max=512,
                 nonlinearity='lrelu',
                 **kwargs):
        
        super().__init__()

        self.fmap_base = fmap_base
        self.fmap_decay = fmap_decay
        self.fmap_max = fmap_max
        
        resolution_log2 = int(np.log2(resolution))
        assert resolution == 2 ** resolution_log2 and resolution >= 4
        self.depth = resolution_log2 - 1 

        self.num_layers = (resolution_log2 - 1) * 2
        self.num_styles = self.num_layers

        act, gain = {'relu': (torch.relu, np.sqrt(2)),
                     'lrelu': (nn.LeakyReLU(negative_slope=0.2), np.sqrt(2))}[nonlinearity]

        
        # Initialize ConstantInputBlock, use self._get_fmaps(1) for channels.
        self.init_block = ...
        
        # We need a list of 1x1 convolutions to convert features to RGB at EACH resolution stage.
        # Initialize the first RGB converter for 4x4 resolution
        rgb_converters = [EqualizedConv2d(...)]

        blocks = []
        for res in range(3, resolution_log2 + 1):
            last_channels = self._get_fmaps(res - 2)
            channels = self._get_fmaps(res - 1)
            
            # Create and append a new SynthesisBlock for this resolution
            blocks.append(SynthesisBlock(...))
            
            # Create and append a new RGB converter for this resolution
            rgb_converters.append(EqualizedConv2d(...))

        self.blocks = nn.ModuleList(blocks)
        self.to_rgb = nn.ModuleList(rgb_converters)

    def _get_fmaps(self, stage):
        return min(
            int(self.fmap_base / (2.0 ** (stage * self.fmap_decay))), 
            self.fmap_max
        )

    def forward(self, dlatents_in, depth=0, alpha=1.0):

        assert depth < self.depth, "Requested output depth cannot be produced"

        # Always run the 4x4 block first. It consumes W vectors at indices [0, 1]
        x = self.init_block(dlatents_in[:, 0:2])
        
        if depth > 0:
            # Run through all blocks that are BELOW the current new depth.
            # These blocks are already fully trained.
            for i, block in enumerate(self.blocks[:depth - 1]):
                x = block(x, dlatents_in[:, 2 * (i + 1) : 2 * (i + 2)])
            
            # Upsample x (nearest) and apply the ToRGB layer from [depth-1]
            residual = ...
            residual = ...

            # Apply the new SynthesisBlock and the new ToRGB layer
            straight = ...
            straight = ...
            
            # Mix straight and residual using alpha
            images_out = ...
        else:
            # Depth 0 (4x4 only)
            images_out = self.to_rgb[0](x)
             
        return images_out

### Задание 13: Generator (1 балл)

Генератор в `StyleGAN` состоит из двух основных подсетей:

- **`Mapping Network`**: Преобразует входной вектор $\mathbf{z}$ в промежуточный латентный вектор $\mathbf{w}$
- **`Synthesis Network`**: Преобразует $\mathbf{w}$ в изображение

#### Style Mixing 

Одной из главных идей `StyleGAN` является **регуляризация с помощью смешивания** (**Mixing Regularization**).

Если генератор всегда обучается, используя только один вектор $\mathbf{w}$ для всех слоев, он может найти ложные взаимосвязи между признаками разного уровня. Например, он может решить, длинные волосы всегда должны быть вместе с женским лицом. Из-за этой жесткой привязки становится невозможно изменить один признак, не задев остальные.

Чтобы заставить сеть выучить независимые стили, авторы предложили использовать следующий трюк во время обучения:

- Генерируем два случайных латентных вектора $\mathbf{z}_1$ и $\mathbf{z}_2$

- Пропускаем их через `Mapping Network` и получаем $\mathbf{w}_1$ и $\mathbf{w}_2$

- Выбираем случайный номер слоя в сети синтеза.

- Для слоев до этого номера используем $\mathbf{w}_1$, а после — $\mathbf{w}_2$.

Это заставляет сеть понимать, что стиль на уровне $4\times 4$ (форма лица) не должен зависеть от стиля на уровне $64\times 64$ (цвет кожи).

Ваша задача — реализовать логику **Style Mixing** внутри метода `forward`.

In [ ]:
class Generator(nn.Module):
    """
    Сlass for the StyleGAN Generator.

    Args:
        resolution: Target output resolution
        latent_size: Dimensionality of input latent Z.
        dlatent_size: Dimensionality of intermediate latent W.
        truncation_psi: Truncation threshold
        truncation_cutoff: Number of layers for which truncation is applied.
        dlatent_avg_beta: Decay for tracking the moving average of W.
        style_mixing_prob: Probability of mixing styles during training.
    """
    def __init__(self, 
                 resolution, 
                 latent_size=512, 
                 dlatent_size=512,
                 truncation_psi=0.7,
                 truncation_cutoff=8, 
                 dlatent_avg_beta=0.995,
                 style_mixing_prob=0.9, 
                 **kwargs):

        super().__init__()
        
        self.style_mixing_prob = style_mixing_prob

        # Calculate total number of layers in the synthesis network
        self.num_layers = (int(np.log2(resolution)) - 1) * 2

        self.mapping_network = MappingNetwork(
            latent_size=latent_size,
            dlatent_size=dlatent_size, 
            num_layers_broadcast=self.num_layers,
            **kwargs 
        )
        
        self.synthesis_network = SynthesisNetwork(
            resolution=resolution, 
            dlatent_size=dlatent_size,
            **kwargs
        )

        if truncation_psi > 0:
            self.truncation = Truncation(
                avg_latent=torch.zeros(dlatent_size),
                max_layer=truncation_cutoff,
                threshold=truncation_psi,
                beta=dlatent_avg_beta
            )
        else:
            self.truncation = None

    def forward(self, latents_in, depth, alpha):
        
        # Pass input latents through the mapping network
        dlatents_in = ...

        if self.training:
            # Update moving average for truncation
            if self.truncation is not None:
                 avg_w = dlatents_in[:, 0].mean(dim=0).detach()
                 self.truncation.update(avg_w.to(self.truncation.avg_latent.device))

            # Style Mixing Regularization
            if self.style_mixing_prob is not None and self.style_mixing_prob > 0:
                # Create a second random batch of latents
                latents2 = torch.randn_like(latents_in)
                # Get w vectors for the second batch
                dlatents2 = ...

                # Create layer indices for masking: [1, Num_Layers, 1]
                layer_idx = ...

                # Pick a random crossover point
                # We mix styles only up to the current depth
                cur_layers = 2 * (depth + 1)
                
                # With probability 'style_mixing_prob', pick a random cutoff
                # Otherwise, use all layers (no mixing)
                mixing_cutoff = random.randint(1, cur_layers) if random.random() < self.style_mixing_prob else cur_layers
                
                # Mix dlatents_in and dlatents2 based on layer_idx and mixing_cutoff
                # Where layer < cutoff, use dlatents_in, else use dlatents2
                dlatents_in = torch.where(..., ..., ...)

            # Apply Truncation Trick 
            if self.truncation is not None:
                dlatents_in = ...

        # Pass the final w styles to the synthesis network
        fake_images = ...

        return fake_images

### Задание 14: Discriminator (1 балл)

**Дискриминатор** (**критик**) в `StyleGAN` архитектурно является противоположностью генератора.

Если генератор увеличивает разрешение изображения, то дискриминатор выполняет обратную операцию. Он принимает готовое RGB-изображение и последовательно сжимает его (**downsampling**), пока не получит финальный скор.

Когда мы добавляем новый слой высокого разрешения, мы так же, как и в генераторе, используем плавный переход (**Fade-in**) с коэффициентом $\alpha$.

Обратите, что для последнего прохода мы используем `DiscriminatorOutputBlock`.

В этом задании вам нужно реализовать архитектуру дискриминатора.

In [ ]:
class Discriminator(nn.Module):
    """
    Discriminator progressively downsamples the image to a single score

    Args:
            resolution: Input image resolution
            num_channels: Number of input color channels
            fmap_base: Base number of feature maps
            fmap_decay: Decay rate for feature maps
            fmap_max: Maximum number of feature maps
            nonlinearity: Activation function name
            mbstd_group_size: Group size for Minibatch StdDev layer
            mbstd_num_features: Number of features for Minibatch StdDev layer
    """
    def __init__(self, 
                 resolution, 
                 num_channels=3, 
                 fmap_base=8192, 
                 fmap_decay=1.0, 
                 fmap_max=512,
                 nonlinearity='lrelu', 
                 mbstd_group_size=4,
                 mbstd_num_features=1, 
                 **kwargs): 
        
        super().__init__()

        self.mbstd_num_features = mbstd_num_features
        self.mbstd_group_size = mbstd_group_size
        self.fmap_base = fmap_base
        self.fmap_decay = fmap_decay
        self.fmap_max = fmap_max

        resolution_log2 = int(np.log2(resolution))
        assert resolution == 2 ** resolution_log2 and resolution >= 4
        self.depth = resolution_log2 - 1

        act, gain = {'relu': (torch.relu, np.sqrt(2)),
                     'lrelu': (nn.LeakyReLU(negative_slope=0.2), np.sqrt(2))}[nonlinearity]

        blocks = []
        from_rgb = []
        
        # Loop descending from max resolution down to 8x8
        for res in range(resolution_log2, 2, -1):
            # Calculate channels for current (in) and next smaller (out) resolution
            in_ch = ...
            out_ch = ...
            
            # Append a DiscriminatorBlock (in_ch -> out_ch)
            blocks.append(...)
    
            # Append a FromRGB layer (num_channels -> in_ch)
            # This is an EqualizedConv2d with kernel_size=1
            from_rgb.append(...)
            
        self.blocks = nn.ModuleList(blocks)
        self.from_rgb = nn.ModuleList(from_rgb)

        # Initialize DiscriminatorOutputBlock
        self.final_block = ...
        
        # FromRGB for the final 4x4 block
        self.from_rgb.append(EqualizedConv2d(
            num_channels, self._get_fmaps(2), kernel_size=1, 
            gain=gain
        ))

        # Downsampler for the residual path in forward()
        self.downsample = nn.AvgPool2d(kernel_size=2)

    def _get_fmaps(self, stage):
        return min(
            int(self.fmap_base / (2.0 ** (stage * self.fmap_decay))), 
            self.fmap_max
        )

    def forward(self, images_in, depth, alpha=1.0):
        
        assert depth < self.depth

        if depth > 0:
            # Downsample images_in and apply the OLD FromRGB layer
            residual = ...
            residual = ...

            # Apply NEW FromRGB layer
            straight = ...
            # Apply the NEW DiscriminatorBlock
            straight = ...

            # Mix straight and residual using alpha
            x = ...

            # Run through the remaining stabilized blocks
            for block in self.blocks[(self.depth - depth):]:
                x = ...
        else:
            # Apply the last FromRGB layer directly
            x = ...
        
        # Final Scoring
        scores_out = ...

        return scores_out

### Задание 15: Функция Потерь (0.5 балла)


Хотя в ранних версиях для `CelebA` использовался `WGAN-GP`, золотым стандартом для `StyleGAN` стала комбинация **Non-Saturating Logistic Loss с регуляризацией R1**. Эта функция потерь состоит из двух компонент:

- **Logistic Loss**: Это стандартная бинарная кросс-энтропия, но записанная через функцию $\text{Softplus}(x) = \log(1 + e^x)$ для численной стабильности

- **R1 Regularization**: В отличие от `WGAN-GP`, который штрафует градиенты везде, R1 накладывает штраф только на градиенты Дискриминатора в точках реальных данных:

$$R_1 = \frac{\gamma}{2} \mathbb{E}_{\mathbf{x} \sim p_{data}(\mathbf{x})} [ ||\nabla D(\mathbf{x})||^2 ]$$

Ваша задача — реализовать класс `LogisticGAN`.

In [ ]:
class LogisticGAN(nn.Module):
    def __init__(self, discriminator):
        super().__init__()
        self.dis = discriminator

    def R1Penalty(self, real_img, height, alpha):

        # Enable gradient tracking for the input image
        # We need this because we are calculating the gradient w.r.t. the IMAGE, not weights.
        real_img = real_img.detach().requires_grad_(True)
        
        # Get Discriminator output (logits)
        real_logit = ...
        
        # Calculate Gradients 
        real_grads = torch.autograd.grad(
            outputs=..., 
            inputs=..., 
            grad_outputs=torch.ones_like(real_logit),
            create_graph=True, 
            retain_graph=True
        )[0].view(real_img.size(0), -1)
        
        # Calculate Penalty: sum(grads^2) across pixels, then mean across batch
        r1_penalty = ...
        
        return r1_penalty

    def dis_loss(self, real_samps, fake_samps, height, alpha, r1_gamma=10.0):

        r_preds = self.dis(real_samps, height, alpha)
        f_preds = self.dis(fake_samps, height, alpha)

        # Implement the logistic loss formula for D
        loss = ...

        # R1 Regularization
        if r1_gamma > 0.0:
            r1_penalty = self.R1Penalty(real_samps.detach(), height, alpha)
            # Add weighted penalty to loss
            loss += ...

        return loss

    def gen_loss(self, _, fake_samps, height, alpha):
        f_preds = self.dis(fake_samps, height, alpha)
        
        # Implement the non-saturating generator loss
        return ...

### Training

В этом разделе мы переходим от архитектуры к обучению. Чтобы вы могли сосредоточиться на самой сути `StyleGAN`, код для визуализации, обновления весов и основного цикла обучения уже реализован за вас.

Функции `plot_loss_curves` и `plot_sample_grid` строят графики лоссов и сетку изображений соответственно.

Функия `update_average` реализует механизм **Exponential Moving Average** (**EMA**). Генератор во время обучения ведет себя нестабильно, его веса постоянно изменяются. Чтобы получать качественные картинки, мы используем **"теневую" копию генератора** (`gen_shadow`), веса которой обновляются очень плавно и усредняются по времени. Именно эту версию мы используем для финальной генерации и валидации.

Всю логику обучения мы добавили в класс `StyleGANTrainer`.

In [ ]:
def plot_loss_curves(train_d_losses, train_g_losses):
  
    clear_output(wait=True)
    plt.figure(figsize=(10, 6))
    
    plt.plot(train_d_losses, label='D Loss', alpha=0.7, linewidth=1)
    plt.plot(train_g_losses, label='G Loss', alpha=0.7, linewidth=1)
    
    plt.title("Training Losses")
    plt.xlabel("Batch Iteration")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_sample_grid(generator, fixed_noise, depth, alpha, gen_shadow=None, 
                     is_final_for_depth=False):

    model = gen_shadow
    model.eval()
    
    with torch.no_grad():
        images = model(fixed_noise, depth, alpha).detach().cpu()
    
    current_res = 2 ** (depth + 2)
    
    if images.shape[2] < 256:
        images = F.interpolate(images, size=256, mode='nearest')

    images = (images * 0.5 + 0.5).clamp(0, 1)
    
    grid = make_grid(images, nrow=len(images), padding=2, pad_value=1)
    
    plt.figure(figsize=(15, 4))
    plt.imshow(grid.permute(1, 2, 0).numpy())
    plt.title(f"Res: {current_res}x{current_res} | Depth: {depth} | Alpha: {alpha:.2f}" + 
              (" FINAL" if is_final_for_depth else ""))
    plt.axis('off')
    plt.show()

In [ ]:
def update_average(model_tgt, model_src, beta):
    with torch.no_grad():
        params_src = dict(model_src.named_parameters())
        params_tgt = dict(model_tgt.named_parameters())
        for k in params_src.keys():
            params_tgt[k].data.copy_(
                torch.lerp(params_tgt[k].data, params_src[k].data, 1.0 - beta)
            )
            
class StyleGANTrainer:
    def __init__(self, 
                 resolution, 
                 num_channels, 
                 latent_size,
                 g_args,                   
                 d_args,                   
                 generator_optimizer,
                 discriminator_optimizer,
                 loss_function,
                 d_repeats=1,
                 ema_decay=0.999, 
                 device=None):
        
        self.device = device if device else torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.depth = int(np.log2(resolution)) - 1
        self.latent_size = latent_size
        self.d_repeats = d_repeats
        self.ema_decay = ema_decay
        
        self.gen = Generator(
            resolution=resolution, 
            latent_size=latent_size,
            **g_args).to(self.device)

        self.dis = Discriminator(
            resolution=resolution, 
            num_channels=num_channels,
            **d_args).to(self.device)

        self.gen_optim = generator_optimizer
        self.dis_optim = discriminator_optimizer
        self.loss = loss_function
        
        self.gen_shadow = copy.deepcopy(self.gen).eval()
        self._update_ema(beta=0.0) 

        self.history = {'d_loss': [], 'g_loss': []}
        self.final_grids_history = [] 

    def _update_ema(self, beta):
        update_average(self.gen_shadow, self.gen, beta)

    def _progressive_downsampling(self, real_batch, depth, alpha):
        target_res = 2 ** (depth + 2)
        target_scale = target_res / real_batch.shape[2]
        ds_real_samples = F.interpolate(real_batch, scale_factor=target_scale, mode='area')
        
        if depth > 0 and alpha < 1.0:
            prior_res = 2 ** (depth + 1)
            prior_scale = prior_res / real_batch.shape[2]
            prior_ds_real_samples = F.interpolate(real_batch, scale_factor=prior_scale, mode='area')
            prior_ds_real_samples = F.interpolate(prior_ds_real_samples, scale_factor=2, mode='nearest')
            real_samples = (alpha * ds_real_samples) + ((1 - alpha) * prior_ds_real_samples)
        else:
            real_samples = ds_real_samples
        return real_samples.to(self.device)

    def _train_step_discriminator(self, noise, real_batch, depth, alpha):
        self.gen.eval()
        self.dis.train()
        loss_val = 0
        for _ in range(self.d_repeats):
            real_samples_ds = self._progressive_downsampling(real_batch, depth, alpha)
            with torch.no_grad():
                fake_samples = self.gen(noise, depth, alpha).detach()
            
            loss = self.loss.dis_loss(real_samples_ds, fake_samples, depth, alpha)
            
            self.dis_optim.zero_grad()
            loss.backward()
            self.dis_optim.step()
            loss_val += loss.item()
        return loss_val / self.d_repeats

    def _train_step_generator(self, noise, real_batch, depth, alpha):
        self.gen.train()
        self.dis.eval()
        real_samples_ds = self._progressive_downsampling(real_batch, depth, alpha)
        fake_samples = self.gen(noise, depth, alpha)
        
        loss = self.loss.gen_loss(real_samples_ds, fake_samples, depth, alpha)
        
        self.gen_optim.zero_grad()
        loss.backward()
        # Gradient Clipping
        nn.utils.clip_grad_norm_(self.gen.parameters(), max_norm=10.)
        self.gen_optim.step()
        
        self._update_ema(beta=self.ema_decay)
        return loss.item()

    def train(self, 
              train_loader, 
              epochs_per_depth, 
              fade_in_percentage, 
              output_dir, 
              num_samples=5, 
              start_depth=0, 
              checkpoint_factor=1):
        
        print("Starting the training process...")
        
        self.gen.train()
        self.dis.train()
        self.gen_shadow.train() 
            
        fixed_noise = torch.randn(num_samples, self.latent_size).to(self.device)
        
        for current_depth in range(start_depth, self.depth):
            current_res = 2 ** (current_depth + 2)
            print(f"\n Training depth: {current_depth} ({current_res}x{current_res}) ")
            
            total_batches = len(train_loader)
            ticker = 1
            current_epochs = epochs_per_depth[current_depth]
            current_fade_in = fade_in_percentage[current_depth]
            
            for epoch in range(1, current_epochs + 1):
                fade_point = int((current_fade_in / 100) * current_epochs * total_batches)
                
                pbar = tqdm(train_loader, initial=0, leave=True, unit="batch")
                pbar.set_description(f"Depth {current_depth} | Epoch {epoch}/{current_epochs}")

                for i, batch_data in enumerate(pbar, 1):
                    real_batch = batch_data[0] if isinstance(batch_data, list) else batch_data
                    
                    alpha = ticker / fade_point if ticker <= fade_point else 1.0
                    
                    real_batch = real_batch.to(self.device)
                    noise = torch.randn(real_batch.size(0), self.latent_size).to(self.device)
                    
                    dis_loss = self._train_step_discriminator(noise, real_batch, current_depth, alpha)
                    gen_loss = self._train_step_generator(noise, real_batch, current_depth, alpha)
                    
                    self.history['d_loss'].append(dis_loss)
                    self.history['g_loss'].append(gen_loss)

                    pbar.set_postfix({
                        'G': f'{gen_loss:.3f}',
                        'D': f'{dis_loss:.3f}',
                        'Alpha': f'{alpha:.2f}'
                    })
                    
                    ticker += 1
                
                print(f"Epoch {epoch} Done.")
                
                plot_loss_curves(
                    self.history['d_loss'], 
                    self.history['g_loss'], 
                )

                if self.final_grids_history:
                    print("History of previous depths")
                    for i, (saved_grid, saved_res) in enumerate(self.final_grids_history):
                        plt.figure(figsize=(15, 2))
                        plt.imshow(saved_grid.permute(1, 2, 0).numpy())
                        plt.title(f"Completed Depth {i} ({saved_res}x{saved_res})")
                        plt.axis('off')
                        plt.show()

                is_last_epoch = (epoch == current_epochs)
                print(f"Current Depth {current_depth}")
                plot_sample_grid(
                    self.gen, fixed_noise, current_depth, alpha, 
                    use_ema=True, gen_shadow=self.gen_shadow,
                    is_final_for_depth=is_last_epoch
                )
                
                if is_last_epoch:
                    with torch.no_grad():
                        self.gen_shadow.eval()
                        imgs = self.gen_shadow(fixed_noise, current_depth, alpha).detach().cpu()
                        if imgs.shape[2] < 256: 
                            imgs = F.interpolate(imgs, size=256, mode='nearest')
                        imgs = (imgs * 0.5 + 0.5).clamp(0, 1)
                        grid = make_grid(imgs, nrow=len(imgs), padding=2, pad_value=1)
                        self.final_grids_history.append((grid, 2**(current_depth+2)))

                if epoch % checkpoint_factor == 0 or is_last_epoch:
                    self.save_checkpoint(output_dir, current_depth, epoch)
                        
        print("Training completed!")

    def save_checkpoint(self, output_dir, depth, epoch):
        save_dir = os.path.join(output_dir, 'models')
        os.makedirs(save_dir, exist_ok=True)
        
        suffix = f"depth_{depth}_epoch_{epoch}.pth"
        torch.save(self.gen.state_dict(), os.path.join(save_dir, f"gen_{suffix}"))
        torch.save(self.dis.state_dict(), os.path.join(save_dir, f"dis_{suffix}"))
        torch.save(self.gen_optim.state_dict(), os.path.join(save_dir, f"gen_optim_{suffix}"))
        torch.save(self.dis_optim.state_dict(), os.path.join(save_dir, f"dis_optim_{suffix}"))
        torch.save(self.gen_shadow.state_dict(), os.path.join(save_dir, f"gen_shadow_{suffix}"))
        
        print(f"Checkpoint is saved: {save_dir}/...{suffix}")

    def load_checkpoint(self, checkpoint_dir, depth, epoch, load_optim=True):
        print(f"Loading checkpoint from: {checkpoint_dir}")
        suffix = f"depth_{depth}_epoch_{epoch}.pth"
        
        self.gen.load_state_dict(torch.load(os.path.join(checkpoint_dir, f"gen_{suffix}"), map_location=self.device))
        self.dis.load_state_dict(torch.load(os.path.join(checkpoint_dir, f"dis_{suffix}"), map_location=self.device))
        
        if load_optim:
            try:
                self.gen_optim.load_state_dict(torch.load(os.path.join(checkpoint_dir, f"gen_optim_{suffix}"), map_location=self.device))
                self.dis_optim.load_state_dict(torch.load(os.path.join(checkpoint_dir, f"dis_optim_{suffix}"), map_location=self.device))
            except FileNotFoundError:
                print("The optimizer files were not found. Skip the loading.")
        

        self.gen_shadow.load_state_dict(torch.load(os.path.join(checkpoint_dir, f"gen_shadow_{suffix}"), map_location=self.device))
        
        print("Checkpoint was uploaded")

In [ ]:
RESOLUTION = 128
LATENT_SIZE = 512
D_LATENT_SIZE = 512
NUM_CHANNELS = 3

EPOCHS_PER_DEPTH = [4, 8, 12, 16, 24, 36] 
FADE_IN_PERCENTAGE = [50, 50, 50, 50, 50, 50]

G_LR = 2e-3
D_LR = 2e-3
BETAS = (0.0, 0.99)
D_REPEATS = 1
EMA_DECAY = 0.999

g_args = {
    'mapping_layers': 8,
    'fmap_base': 8192,
    'fmap_decay': 1.0,
    'fmap_max': 512,
}
d_args = {
    'mbstd_group_size': 4,
    'mbstd_num_features': 1,
    'fmap_base': 8192,
    'fmap_decay': 1.0,
    'fmap_max': 512,
}

OUTPUT_DIR = "stylegan_checkpoints"
DEVICE = 'cuda'

In [ ]:
gen = Generator(RESOLUTION, LATENT_SIZE, D_LATENT_SIZE, **g_args)
dis = Discriminator(RESOLUTION, NUM_CHANNELS, **d_args)

g_optim = optim.Adam(gen.parameters(), lr=G_LR, betas=BETAS)
d_optim = optim.Adam(dis.parameters(), lr=D_LR, betas=BETAS)

loss_fn = LogisticGAN(dis)

In [ ]:
trainer = StyleGANTrainer(
    resolution=RESOLUTION,
    num_channels=NUM_CHANNELS,
    latent_size=LATENT_SIZE,
    g_args=g_args,
    d_args=d_args,
    
    generator_optimizer=g_optim,
    discriminator_optimizer=d_optim,
    loss_function=loss_fn,
    
    d_repeats=D_REPEATS,
    ema_decay=EMA_DECAY,
    device=DEVICE
)

trainer.gen_optim.param_groups[0]['params'] = list(trainer.gen.parameters())
trainer.dis_optim.param_groups[0]['params'] = list(trainer.dis.parameters())
trainer.loss.dis = trainer.dis 

In [ ]:
trainer.train(
    train_loader=train_loader,  
    epochs_per_depth=EPOCHS_PER_DEPTH,
    fade_in_percentage=FADE_IN_PERCENTAGE,
    output_dir=OUTPUT_DIR,
    num_samples=5,
    start_depth=0,
    checkpoint_factor=2
)

### Задание 16, бонус: Style Mixing (1 балл)

Одной из самых главных возможностей `StyleGAN` является **смешивание стилей** (**Style Mixing**). Поскольку мы знаем, что разные слои сети синтеза отвечают за разные уровни детализации, мы можем создавать новые изображения, комбинируя векторы стиля $\mathbf{w}$ от двух разных источников.

Эксперимент строится следующим образом:

- **Source A**: Генерирует вектор $\mathbf{w}_A$ и определяет основу изображения.

- **Source B**: Генерирует вектор $\mathbf{w}_B$ и вносит свои атрибуты.

Мы создаем новый, смешанный вектор $\mathbf{w}_{mix}$, который состоит из $\mathbf{w}_A$, но в определенных слоя мы заменяем его на $\mathbf{w}_B$.

Уровни смешивания:

- **Coarse**: Замена $\mathbf{w}_A$ на $\mathbf{w}_B$ в слоях низкого разрешения ($4 \times 4 - 8 \times 8$)

- **Middle**: Замена в слоях среднего разрешения ($16 \times 16 - 32 \times 32$)

- **Fine**: Замена в слоях высокого разрешения ($64 \times 64 - 128 \times 128$)

Вам нужно заполнить пропуски в функциях `mix_styles` и `visualize_style_mixing`.

In [ ]:
def mix_styles(w_batch_A, w_batch_B, style_range):
    """
    Creates a mixed W tensor by copying styles from B into A for specified layers.
    Args:
        w_batch_A: W tensor from Source A
        w_batch_B: W tensor from Source B
        style_range: List or range of layer indices to swap 
    """
    w_mix = w_batch_A.clone()
    
    # Perform the style swap using slicing
    w_mix[:, ..., :] = ...
    
    return w_mix

In [ ]:
def visualize_style_mixing(generator, 
                           n_source_A=5, 
                           n_source_B=3,
                           style_type='coarse',
                           device='cuda',
                           seed=42):

    torch.manual_seed(seed)
    
    if style_type == 'coarse':
        style_layers = range(0, 4)   # 4x4, 8x8
    elif style_type == 'middle':
        style_layers = range(4, 8)   # 16x16, 32x32
    elif style_type == 'fine':
        style_layers = range(8, 12)  # 64x64, 128x128
        
    print(f"Generating mixing grid for '{style_type}' styles (layers {list(style_layers)})...")

    with torch.no_grad():
        dim_z = generator.mapping_network.layers[0].weight.shape[1]
        z_A = torch.randn(n_source_A, dim_z).to(device)
        z_B = torch.randn(n_source_B, dim_z).to(device)
        
        # Get w using Mapping Network
        w_A =...
        w_B = ...
        

        max_depth = generator.synthesis_network.depth - 1
        
        # Get images using Synthesis Network
        img_A = ...
        img_B = ...

        rows = []

        empty = torch.ones(1, 3, 128, 128).to(device) 
        header_row = torch.cat([empty, img_A], dim=0)
        rows.append(header_row)
        
        for i in range(n_source_B):
            w_this_B = w_B[i].unsqueeze(0).expand(n_source_A, -1, -1)
            
            w_mixed = mix_styles(generator, w_batch_A=w_A, w_batch_B=w_this_B, style_range=style_layers)
            img_mixed = generator.synthesis_network(w_mixed, depth=max_depth, alpha=1.0)
            
            row = torch.cat([img_B[i].unsqueeze(0), img_mixed], dim=0)
            rows.append(row)
            
        final_grid_tensor = torch.cat(rows, dim=0)
        
        final_grid_tensor = (final_grid_tensor * 0.5 + 0.5).clamp(0, 1)

        grid_img = make_grid(final_grid_tensor, nrow=n_source_A + 1, padding=2, pad_value=1)
        
        plt.figure(figsize=(15, 10))
        plt.imshow(grid_img.permute(1, 2, 0).cpu().numpy())
        plt.axis('off')
        plt.title(f"Style Mixing: {style_type.capitalize()} Styles")
        plt.show()

Используйте реализованные функции, чтобы увидеть результаты смешивания на разных уровнях: `coarse`, `middle` и `fine`.

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ 
# Your code here

In [ ]:
# Здесь можно оставить отзывы, пожелания и впечатления о ДЗ :)